<a href="https://colab.research.google.com/github/dong-uk-kim97/Exploration/blob/main/Exploration_5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 인공지능과 가위바위보

이미지 사이즈 바꾸는 함수 생성 및 적용

In [1]:
from PIL import Image
import  glob
import numpy as np
import matplotlib.pyplot as plt


def resize_images(img_path):
    images = glob.glob(img_path + '/*.jpg')
    
    print(len(images), 'images to be resized.')
    
    # 파일마다 모두 28*28 사이즈로 바꾸어 저장
    target_size = (28,28)
    for img in images:
        old_img = Image.open(img)
        new_img = old_img.resize(target_size, Image.ANTIALIAS)
        new_img.save(img, 'JPEG')
        
    print(len(images), 'images resized')
    
# 가위 이미지가 저장된 디렉토리 아래의 모든 jpg 파일을 읽어들여서
image_dir_path = "/content/drive/MyDrive/rock_scissor_paper/scissor1"
resize_images(image_dir_path)

# 바위 이미지
image_dir_path = "/content/drive/MyDrive/rock_scissor_paper/rock1"
resize_images(image_dir_path)

# 보 이미지
image_dir_path = "/content/drive/MyDrive/rock_scissor_paper/paper1"
resize_images(image_dir_path)

100 images to be resized.
100 images resized
100 images to be resized.
100 images resized
100 images to be resized.
100 images resized


이미지 사진 불러오기

In [2]:
# 가위, 바위, 보 데이터를 읽을 수 있는 load_data 함수 만들기
# 함수 입력으로 이미지 폴더 위치 받음
# 3개의 클래스로 라벨링(가위:0, 바위:1, 보:2)import numpy as np

def load_data(img_path, number_of_data=300):  # 가위바위보 이미지 개수 총합에 주의하세요.
    # 가위 : 0, 바위 : 1, 보 : 2
    img_size=28
    color=3
    #이미지 데이터와 라벨(가위 : 0, 바위 : 1, 보 : 2) 데이터를 담을 행렬(matrix) 영역을 생성합니다.
    imgs=np.zeros(number_of_data*img_size*img_size*color,dtype=np.int32).reshape(number_of_data,img_size,img_size,color)
    labels=np.zeros(number_of_data,dtype=np.int32)

    idx=0
    for file in glob.iglob(img_path+'/scissor1/*.jpg'):
        img = np.array(Image.open(file),dtype=np.int32)
        imgs[idx,:,:,:]=img    # 데이터 영역에 이미지 행렬을 복사
        labels[idx]=0   # 가위 : 0
        idx=idx+1

    for file in glob.iglob(img_path+'/rock1/*.jpg'):
        img = np.array(Image.open(file),dtype=np.int32)
        imgs[idx,:,:,:]=img    # 데이터 영역에 이미지 행렬을 복사
        labels[idx]=1   # 바위 : 1
        idx=idx+1  
    
    for file in glob.iglob(img_path+'/paper1/*.jpg'):
        img = np.array(Image.open(file),dtype=np.int32)
        imgs[idx,:,:,:]=img    # 데이터 영역에 이미지 행렬을 복사
        labels[idx]=2   # 보 : 2
        idx=idx+1
        
    print("데이터의 이미지 개수는", idx,"입니다.")
    return imgs, labels

train 데이터 표준화

In [3]:
# train data
image_dir_path = "/content/drive/MyDrive/rock_scissor_paper"
(x_train, y_train) = load_data(image_dir_path)

# 입력 정규화(0~1)
x_train_norm = x_train/255.0
x_train_reshaped=x_train_norm.reshape(-1, 28, 28, 3) 

print("x_train.shape:{}".format(x_train.shape))
print("y_train.shape:{}".format(y_train.shape))

데이터의 이미지 개수는 300 입니다.
x_train.shape:(300, 28, 28, 3)
y_train.shape:(300,)


모델 생성 및 학습

In [174]:
import tensorflow as tf
from tensorflow import keras
import numpy as np

model=keras.models.Sequential()
model.add(keras.layers.Conv2D(32, (3,3), activation='relu', padding = 'same',  input_shape=(28,28,3)))
model.add(keras.layers.MaxPool2D(2,2))
model.add(keras.layers.Conv2D(64, (3,3), padding = 'same',  activation='relu')) # same padding을 사용하니 조금 더 향상된 점수를 얻을 수 있었다.
model.add(keras.layers.MaxPooling2D((2,2)))
model.add(keras.layers.MaxPooling2D((2,2)))
model.add(keras.layers.Flatten())
# model.add(keras.layers.Dropout(0.99)) # 0.1과 0.99와의 차이가 무의미해서 넣지 않는다.
model.add(keras.layers.Dense(50, activation='relu'))
model.add(keras.layers.Dense(3, activation='softmax')) # 우리가 원하는 값은 가위,바위,보 3가지이기 때문에 출력층을 3개로 하고 활성화함수로 'softmax'함수를 사용한다.

model.summary()

Model: "sequential_24"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_48 (Conv2D)          (None, 28, 28, 32)        896       
                                                                 
 max_pooling2d_68 (MaxPoolin  (None, 14, 14, 32)       0         
 g2D)                                                            
                                                                 
 conv2d_49 (Conv2D)          (None, 14, 14, 64)        18496     
                                                                 
 max_pooling2d_69 (MaxPoolin  (None, 7, 7, 64)         0         
 g2D)                                                            
                                                                 
 max_pooling2d_70 (MaxPoolin  (None, 3, 3, 64)         0         
 g2D)                                                            
                                                     

In [175]:
model.compile(optimizer='adam',
             loss='sparse_categorical_crossentropy',
             metrics=['accuracy'])

model.fit(x_train_reshaped, y_train, epochs=20)

Epoch 1/20
10/10 [==============================] - 0s 3ms/step - loss: 1.1186 - accuracy: 0.2967
Epoch 2/20
10/10 [==============================] - 0s 3ms/step - loss: 1.0993 - accuracy: 0.3467
Epoch 3/20
10/10 [==============================] - 0s 3ms/step - loss: 1.0995 - accuracy: 0.3667
Epoch 4/20
10/10 [==============================] - 0s 3ms/step - loss: 1.0945 - accuracy: 0.3267
Epoch 5/20
10/10 [==============================] - 0s 3ms/step - loss: 1.0990 - accuracy: 0.3333
Epoch 6/20
10/10 [==============================] - 0s 3ms/step - loss: 1.1002 - accuracy: 0.3200
Epoch 7/20
10/10 [==============================] - 0s 3ms/step - loss: 1.0967 - accuracy: 0.3133
Epoch 8/20
10/10 [==============================] - 0s 3ms/step - loss: 1.0910 - accuracy: 0.3567
Epoch 9/20
10/10 [==============================] - 0s 4ms/step - loss: 1.0879 - accuracy: 0.3300
Epoch 10/20
10/10 [==============================] - 0s 3ms/step - loss: 1.0768 - accuracy: 0.5400
Epoch 11/20
10/10 [

In [176]:
def resize_image(img_path):
    images = glob.glob(img_path + '/*.jpg')
    
    print(len(images), 'images to be resized.')
    
    # 파일마다 모두 28*28 사이즈로 바꾸어 저장
    target_size = (28,28)
    for img in images:
        old_img = Image.open(img)
        new_img = old_img.resize(target_size, Image.ANTIALIAS)
        new_img.save(img, 'JPEG')
        
    print(len(images), 'images resized')
    
# 가위 이미지가 저장된 디렉토리 아래의 모든 jpg 파일을 읽어들여서
image_dir_path = "/content/drive/MyDrive/rock_scissor_paper/test/scissor"
resize_image(image_dir_path)

# 바위 이미지
image_dir_path = "/content/drive/MyDrive/rock_scissor_paper/test/rock"
resize_image(image_dir_path)

# 보 이미지
image_dir_path = "/content/drive/MyDrive/rock_scissor_paper/test/paper"
resize_image(image_dir_path)

100 images to be resized.
100 images resized
100 images to be resized.
100 images resized
100 images to be resized.
100 images resized


In [177]:
def load_datas(img_path, number_of_data=300):  # 가위바위보 이미지 개수 총합에 주의하세요.
    # 가위 : 0, 바위 : 1, 보 : 2
    img_size=28
    color=3
    #이미지 데이터와 라벨(가위 : 0, 바위 : 1, 보 : 2) 데이터를 담을 행렬(matrix) 영역을 생성합니다.
    imgs=np.zeros(number_of_data*img_size*img_size*color,dtype=np.int32).reshape(number_of_data,img_size,img_size,color)
    labels=np.zeros(number_of_data,dtype=np.int32)

    idx=0
    for file in glob.iglob(img_path+'/scissor/*.jpg'):
        img = np.array(Image.open(file),dtype=np.int32)
        imgs[idx,:,:,:]=img    # 데이터 영역에 이미지 행렬을 복사
        labels[idx]=0   # 가위 : 0
        idx=idx+1

    for file in glob.iglob(img_path+'/rock/*.jpg'):
        img = np.array(Image.open(file),dtype=np.int32)
        imgs[idx,:,:,:]=img    # 데이터 영역에 이미지 행렬을 복사
        labels[idx]=1   # 바위 : 1
        idx=idx+1  
    
    for file in glob.iglob(img_path+'/paper/*.jpg'):
        img = np.array(Image.open(file),dtype=np.int32)
        imgs[idx,:,:,:]=img    # 데이터 영역에 이미지 행렬을 복사
        labels[idx]=2   # 보 : 2
        idx=idx+1
        
    print("데이터의 이미지 개수는", idx,"입니다.")
    return imgs, labels

In [178]:
img_dir_path = "/content/drive/MyDrive/rock_scissor_paper/test"
(x_test, y_test) = load_datas(img_dir_path)
x_test_norm = x_test/255.0
x_test_reshaped=x_test_norm.reshape(-1, 28, 28, 3)


print('x_test.shape :{}'.format(x_test_norm.shape))
print('y_test.shape :{}'.format(y_test.shape))

데이터의 이미지 개수는 300 입니다.
x_test.shape :(300, 28, 28, 3)
y_test.shape :(300,)


In [179]:
test_loss, test_accuracy = model.evaluate(x_test_reshaped, y_test, verbose=2)

print('test_loss :{}'.format(test_loss))
print('test_accuracy :{}'.format(test_accuracy))

10/10 - 0s - loss: 0.7766 - accuracy: 0.7333 - 129ms/epoch - 13ms/step
test_loss :0.7765754461288452
test_accuracy :0.7333333492279053


# 회고

길고 긴 싸움이었다. 정확도가 돌릴 때마다 요동쳐서 이거에 대한 원인 분석이 시급한 것 같다. 그래도 결과물은 있었다. same padding을 사용함으로써 정확도를 조금 올렸던 것 같고, maxpooling을 두 번 했더니 값이 더 괜찮게 나왔다. 조금 더 공부해서 이 부분에 대한 분석을 해야겠다. 그리고 같은 데이터로 남들과는 좋지 못한 정확도가 나와서 당황스러웠다. 아마도 모델 설계를 잘못한 거 같은데 이를 분석하는게 나의 과제로 남아있다.